In [1]:
import pandas as pd
import numpy as np
import pandas as pd
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import math 
import seaborn as sns
import matplotlib.colors as mcolors
import statsmodels.api as sm
import statsmodels.formula.api as smf
from statsmodels.formula.api import ols
from statsmodels.formula.api import mixedlm
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
import os
import matplotlib.pyplot as mpl
import matplotlib

In [2]:
parentDirectory = os.path.abspath(os.path.join(os.path.join(os.getcwd(), os.pardir), os.pardir))
DATA_DIR = parentDirectory +'/data/'
FIGURES_DIR = parentDirectory +'/figures/'

def make_stars(val):
    '''
    if val<0.0001:
        return '****'
    elif val<0.001:
        return '***'
    elif val<0.01:
        return '**'
    '''
    if val<0.05:
        return '*'
    else:
        return ''
    
def generate_equation(order):
    if order == 'Cubic':
        eq = "volume_total ~ intervention_flag*k*year + intervention_flag*np.power(k,2)*year + intervention_flag*np.power(k,3)*year"
    elif order == "Quadratic":
        eq = "volume_total ~ intervention_flag*k*year + intervention_flag*np.power(k,2)*year"
    elif order == "Linear":
        eq = "volume_total ~ intervention_flag*k*year"
    elif order == 'Constant':
        eq = "volume_total ~ intervention_flag*year"
    return eq

## food entities

In [3]:
df = pd.read_parquet(DATA_DIR+'dk_new_food_timeseries_items1.parquet')
df = df.loc[df['name']!='Bánh mì']

full_names = {
    'AU': 'Australia',
    'BR': 'Brazil',
    'CA': 'Canada',
    'FR': 'France',
    'DE': 'Germany',
    'IN': 'India',
    'IT': 'Italy',
    'MX': 'Mexico',
    'ES': 'Spain',
    'GB': 'United Kingdom',
    'US': 'United States',
    'DK': 'Denmark'
}

event_dicts = [{'country': 'AU',
  'start_md_1': '2020-03-27',
  'end_md_1': '2020-06-07',
  'start_md_2': np.nan},
 {'country': 'BR',
  'start_md_1': '2020-03-23',
  'end_md_1': '2020-08-09',
  'start_md_2': np.nan},
 {'country': 'CA',
  'start_md_1': '2020-03-19',
  'end_md_1': '2020-06-21',
  'start_md_2': '2020-10-12'},
 {'country': 'DE',
  'start_md_1': '2020-03-21',
  'end_md_1': '2020-05-09',
  'start_md_2': '2020-12-18'},
 {'country': 'DK',
  'start_md_1': '2020-03-17',
  'end_md_1': '2020-05-07',
  'start_md_2': np.nan},
 {'country': 'ES',
  'start_md_1': '2020-03-17',
  'end_md_1': '2020-06-14',
  'start_md_2': '2020-11-07'},
 {'country': 'FR',
  'start_md_1': '2020-03-18',
  'end_md_1': '2020-06-08',
  'start_md_2': '2020-11-01'},
 {'country': 'GB',
  'start_md_1': '2020-03-23',
  'end_md_1': '2020-08-03',
  'start_md_2': '2020-10-21'},
 {'country': 'IN',
  'start_md_1': '2020-03-24',
  'end_md_1': '2020-10-29',
  'start_md_2': np.nan},
 {'country': 'IT',
  'start_md_1': '2020-03-11',
  'end_md_1': '2020-06-06',
  'start_md_2': '2020-11-06'},
 {'country': 'JP',
  'start_md_1': '2020-04-12',
  'end_md_1': '2020-05-30',
  'start_md_2': np.nan},
 {'country': 'KE',
  'start_md_1': '2020-03-24',
  'end_md_1': '2020-10-04',
  'start_md_2': np.nan},
 {'country': 'MX',
  'start_md_1': '2020-03-25',
  'end_md_1': '2020-10-06',
  'start_md_2': np.nan},
 {'country': 'NG',
  'start_md_1': '2020-03-27',
  'end_md_1': '2020-08-09',
  'start_md_2': np.nan},
 {'country': 'US',
  'start_md_1': '2020-03-21',
  'end_md_1': '2020-06-11',
  'start_md_2': '2020-11-26'}]

df_events = pd.DataFrame(event_dicts)
df_events['start_md_1'] = pd.to_datetime(df_events['start_md_1'])
df_events['end_md_1'] = pd.to_datetime(df_events['end_md_1'])
df_events['start_md_2'] = pd.to_datetime(df_events['start_md_2'])

df_agg = pd.read_pickle(DATA_DIR+'df_agg_cats.pickle')

df = df.loc[df['country_code'].isin(full_names.keys())]
df_agg = df_agg.loc[df_agg['country'].isin(full_names.keys())]

weeks_2019 = list(df_agg.iloc[0]['volume_weekly_total'].index)[:52]
weeks_2020 = list(df_agg.iloc[0]['volume_weekly_total'].index)[52:]

smoothing_term = np.power(10.0,-10)

In [4]:
def short_term_estimate(row, country):
    l = []
    start_md = df_events.loc[df_events['country'] == country].iloc[0]['start_md_1']
    end_md = df_events.loc[df_events['country'] == country].iloc[0]['end_md_1']
    start_md2 = df_events.loc[df_events['country'] == country].iloc[0]['start_md_2']

    for week in zip(row.keys(),row.values()):

        entry = {}

        if week[0] in weeks_2020:
            date = pd.to_datetime(week[0])

            if type(start_md2)!=pd._libs.tslibs.nattype.NaTType and date > start_md2:
                continue

            entry['k'] = math.floor(((date - start_md).days +7) / 7)
            entry['volume_total'] = week[1]
            entry['year'] = '2020'
            l.append(entry)

        elif week[0] in weeks_2019:
            date = pd.to_datetime(weeks_2020[weeks_2019.index(week[0])])

            if type(start_md2)!=pd._libs.tslibs.nattype.NaTType and date > start_md2:
                continue

            entry['k'] = math.floor(((date - start_md).days +7) / 7)
            entry['volume_total'] = week[1]
            entry['year'] = '2019'
            l.append(entry)

    df_totals = pd.DataFrame(l)

    k = 30
    df_totals['intervention_flag'] = df_totals['k'].apply(lambda x: 1 if x >= 0 else 0)
    df_temp = df_totals.loc[(df_totals['k'] >= -k) & (df_totals['k'] <= k)].copy()
    df_temp['volume_total'] = df_temp['volume_total'].apply(lambda x: np.log(x+smoothing_term))

    mod = smf.ols(generate_equation('Quadratic'), data = df_temp)
    result = mod.fit(cov_type='hc0')

    alpha = 'intervention_flag:year[T.2020]'

    return np.exp(result.params[alpha])-1, result.pvalues[alpha]

In [42]:
entry_list = []

c = 0

for category, gr1 in df.groupby('name'):
    c+=1
    if c%50==0:
        print(c,'/',len(df.groupby('name')))
              
    for country, gr2 in gr1.groupby('country_code'):
        
        entry = {}

        entry['name'] = category
        entry['category'] = gr2.iloc[0]['category']
        entry['country'] = country
        entry['avg_weekly_volume'] = np.mean(list(gr2.iloc[0]['ts']['max_ratio'].values()))
        entry['coeff'], entry['p_val'] = short_term_estimate(gr2.iloc[0]['ts']['max_ratio'], country)
        
        entry_list.append(entry)    

50 / 1394
100 / 1394
150 / 1394
200 / 1394
250 / 1394
300 / 1394
350 / 1394
400 / 1394
450 / 1394
500 / 1394
550 / 1394
600 / 1394
650 / 1394
700 / 1394
750 / 1394
800 / 1394
850 / 1394
900 / 1394
950 / 1394
1000 / 1394
1050 / 1394
1100 / 1394
1150 / 1394
1200 / 1394
1250 / 1394
1300 / 1394
1350 / 1394


In [175]:
df_results = pd.DataFrame(entry_list)
df_results = df_results.loc[df_results['avg_weekly_volume']>=0.3].copy(deep = True)

In [184]:
df_results.loc[df_results['name'] == 'Baking powder']

,name,category,country,avg_weekly_volume,coeff,p_val
713,Baking powder,pastry and bakery product,AU,4.535492,0.845078,3.121508e-02
714,Baking powder,pastry and bakery product,BR,11.198266,0.867766,5.484677e-03
715,Baking powder,pastry and bakery product,CA,4.365574,3.172059,1.925057e-06
716,Baking powder,pastry and bakery product,DE,6.858081,1.172940,3.755107e-07
717,Baking powder,pastry and bakery product,DK,2.328068,0.598831,5.668655e-01
718,Baking powder,pastry and bakery product,ES,4.276234,11.827717,4.411832e-05
719,Baking powder,pastry and bakery product,FR,13.271121,11.359965,2.149668e-11
720,Baking powder,pastry and bakery product,GB,5.895561,3.778092,1.137059e-05
721,Baking powder,pastry and bakery product,IN,30.172095,9.973420,5.375998e-08
722,Baking powder,pastry and bakery product,IT,8.606735,11.312144,1.778666e-08


In [176]:
tmp = df_results.groupby('name').apply(lambda x: pd.Series({
    'effect_median': x['coeff'].median(), 
    'category': x['category'].unique()[0],
    'C': x['country'].unique(),
    'exists_all_countries': len(x)==12,
    'all': x[['country','coeff','p_val']]
    })).reset_index()

tmp = tmp.loc[tmp['exists_all_countries']]

In [177]:
tmp.sort_values(by = 'effect_median', ascending =  False).head(10)[['name','effect_median','category']]

,name,effect_median,category
56,Baking powder,3.066004,pastry and bakery product
941,Sourdough,2.346192,bread and flatbread
1055,Turnip,2.325517,vegetable and legume
245,Chocolate chip cookie,1.622576,dessert
357,Empanada,1.477386,"pasta, pizza and noodle dish"
139,Brioche,1.466175,pastry and bakery product
29,Apple pie,1.407466,pie
805,Powdered sugar,1.387046,dessert
309,Crêpe,1.293176,dessert
134,Bread (Literature Subject),1.281436,bread and flatbread


In [178]:
tmp.sort_values(by = 'effect_median', ascending =  True).head(10)[['name','effect_median','category']]

,name,effect_median,category
1007,Tapas,-0.397568,snack
289,Cotton candy,-0.289522,dessert
499,Jackfruit,-0.271534,fruit
4,Absinthe,-0.216345,"wine, beer and liquor"
345,Edamame,-0.192057,snack
214,Champagne,-0.187970,"wine, beer and liquor"
273,Common sage,-0.175960,herb
757,Persimmon,-0.152744,fruit
512,Kale,-0.131959,vegetable and legume
270,Cola,-0.087060,soft drink


In [211]:
for i,row in tmp.sort_values(by = 'effect_median', ascending =  False).head(10)[['name','effect_median','category','all']].iterrows():
    print(row['name']+' ('+row['category']+') & ')
    print('+'+str(round(row['effect_median']*100,2))+'\%' if \
              round(row['effect_median'],2) >=0 else \
              str(round(row['effect_median']*100,2))+'\%',' & ')

    
    #print(row['all']['country'].values)
    #for c,r in row['all'].values().iterrows():
        #
    for i in range(12):
        print('+'+str(round(row['all']['coeff'].values[i]*100,2))+'\%' if \
              round(row['all']['coeff'].values[i],2) >=0 else \
              str(round(row['all']['coeff'].values[i]*100,2))+'\%', make_stars(row['all']['p_val'].values[i]) + ' &')

    print('----------------------------------')
    print('\n')

Baking powder (pastry and bakery product) & 
+306.6\%  & 
+84.51\% * &
+86.78\% * &
+317.21\% * &
+117.29\% * &
+59.88\%  &
+1182.77\% * &
+1136.0\% * &
+377.81\% * &
+997.34\% * &
+1131.21\% * &
+295.99\% * &
+253.09\% * &
----------------------------------


Sourdough (bread and flatbread) & 
+234.62\%  & 
+116.27\% * &
+133.27\% * &
+748.77\% * &
+179.79\% * &
+67.76\%  &
+649.08\% * &
+1579.6\% * &
+276.11\% * &
-98.99\%  &
+249.87\%  &
+54092.48\%  &
+219.36\% * &
----------------------------------


Turnip (vegetable and legume) & 
+232.55\%  & 
+1688.95\% * &
+107.67\%  &
+228.78\% * &
+236.32\% * &
+31932262078.57\%  &
+224.23\% * &
+101.81\% * &
+529.55\% * &
+54.36\% * &
-33.77\%  &
+457.63\% * &
+506.32\% * &
----------------------------------


Chocolate chip cookie (dessert) & 
+162.26\%  & 
+117.41\% * &
+95.09\%  &
+266.66\% * &
+162.33\% * &
+370.66\%  &
+162.18\%  &
+1369.27\%  &
+375.18\% * &
+65.2\%  &
+35.79\%  &
+130.06\%  &
+242.94\% * &
--------------------------

In [212]:
for i,row in tmp.sort_values(by = 'effect_median', ascending =  True).head(10)[['name','effect_median','category','all']].iterrows():
    print(row['name']+' ('+row['category']+') & ')
    print('+'+str(round(row['effect_median']*100,2))+'\%' if \
              round(row['effect_median'],2) >=0 else \
              str(round(row['effect_median']*100,2))+'\%',' & ')

    
    #print(row['all']['country'].values)
    #for c,r in row['all'].values().iterrows():
        #
    for i in range(12):
        print('+'+str(round(row['all']['coeff'].values[i]*100,2))+'\%' if \
              round(row['all']['coeff'].values[i],2) >=0 else \
              str(round(row['all']['coeff'].values[i]*100,2))+'\%', make_stars(row['all']['p_val'].values[i]) + ' &')

    print('----------------------------------')
    print('\n')

Tapas (snack) & 
-39.76\%  & 
-41.02\%  &
+284.01\% * &
-77.24\% * &
-65.98\% * &
-52.68\% * &
-71.2\% * &
-79.11\% * &
-38.49\%  &
+44.82\%  &
-10.61\%  &
+7.13\%  &
-36.25\% * &
----------------------------------


Cotton candy (dessert) & 
-28.95\%  & 
-42.46\%  &
-34.61\% * &
+8.3\%  &
-31.85\%  &
-94.91\%  &
-68.19\%  &
-18.33\%  &
+258.52\% * &
-32.71\%  &
-26.05\%  &
-18.04\%  &
+9.95\%  &
----------------------------------


Jackfruit (fruit) & 
-27.15\%  & 
-30.47\%  &
-23.84\%  &
-30.49\%  &
+91.13\%  &
+4675.13\%  &
-73.95\%  &
-52.86\%  &
-17.13\%  &
-9.74\%  &
-85.1\% * &
+35.67\%  &
-32.84\% * &
----------------------------------


Absinthe (wine, beer and liquor) & 
-21.63\%  & 
-16.57\%  &
-38.14\%  &
-26.69\%  &
+11.81\%  &
+12914.16\%  &
-48.51\%  &
+-0.37\%  &
-40.42\%  &
-56.23\% * &
-28.4\%  &
+28100.37\%  &
+17.72\%  &
----------------------------------


Edamame (snack) & 
-19.21\%  & 
+34.85\%  &
-7.55\%  &
-19.24\%  &
-47.25\% * &
+26.35\%  &
-56.26\%  &
-23.31

## modes entities

In [213]:
df = pd.read_parquet(DATA_DIR+'modes_fine.parquet')
df = df.loc[df['country_code'].isin(full_names.keys())]

In [214]:
entry_list = []

c = 0

for category, gr1 in df.groupby('name'):
    c+=1
    if c%50==0:
        print(c,'/',len(df.groupby('name')))
              
    for country, gr2 in gr1.groupby('country_code'):
        
        entry = {}

        entry['name'] = category
        entry['category'] = gr2.iloc[0]['category']
        entry['country'] = country
        entry['avg_weekly_volume'] = np.mean(list(gr2.iloc[0]['ts']['max_ratio'].values()))
        entry['coeff'], entry['p_val'] = short_term_estimate(gr2.iloc[0]['ts']['max_ratio'], country)
        
        entry_list.append(entry)    

In [217]:
df_results = pd.DataFrame(entry_list)
#df_results = df_results.loc[df_results['avg_weekly_volume']>=0.3].copy(deep = True)

In [219]:
tmp = df_results.groupby('name').apply(lambda x: pd.Series({
    'effect_median': x['coeff'].median(), 
    'category': x['category'].unique()[0],
    'C': x['country'].unique(),
    'exists_all_countries': len(x)==12,
    'all': x[['country','coeff','p_val']]
    })).reset_index()

#tmp = tmp.loc[tmp['exists_all_countries']]

In [221]:
tmp.sort_values(by = 'effect_median', ascending = False)

,name,effect_median,category,C,exists_all_countries,all
15,Take-out,1.545930,Mode 2,"[AU, BR, CA, DE, DK, ES, FR, GB, IN, IT, MX, US]",True,country coeff p_val 179 A...
12,Recipe,0.881785,Mode 1,"[AU, BR, CA, DE, DK, ES, FR, GB, IN, IT, MX, US]",True,country coeff p_val 179 A...
7,Food delivery,0.871674,Mode 2,"[AU, BR, CA, DE, DK, ES, FR, GB, IN, IT, MX, US]",True,country coeff p_val 179 ...
0,Baking,0.842290,Mode 1,"[AU, BR, CA, DE, DK, ES, FR, GB, IN, IT, MX, US]",True,country coeff p_val 179 A...
4,Cooking,0.593776,Mode 1,"[AU, BR, CA, DE, DK, ES, FR, GB, IN, IT, MX, US]",True,country coeff p_val 179 A...
1,Barbecue,0.295275,Mode 4,"[AU, BR, CA, DE, DK, ES, FR, GB, IN, IT, MX, US]",True,country coeff p_val 179 AU 0...
9,Grocery store,0.179572,Mode 1,"[AU, BR, CA, DE, DK, ES, FR, GB, IN, IT, MX, US]",True,country coeff p_val 179 A...
14,Supermarket,0.097571,Mode 1,"[AU, BR, CA, DE, DK, ES, FR, GB, IN, IT, MX, US]",True,country coeff p_val 179 AU -0...
11,Picnic,-0.273105,Mode 4,"[AU, BR, CA, DE, DK, ES, FR, GB, IN, IT, MX, US]",True,country coeff p_val 179 AU -0...
5,Diner,-0.330995,Mode 3,"[AU, BR, CA, DE, DK, ES, FR, GB, IN, IT, MX, US]",True,country coeff p_val 179 AU ...


In [225]:
for i,row in tmp.sort_values(by = 'effect_median', ascending =  False)[['name','effect_median','category','all']].iterrows():
    print(row['name']+' ('+row['category']+') & ')
    print('+'+str(round(row['effect_median']*100,2))+'\%' if \
              round(row['effect_median'],2) >=0 else \
              str(round(row['effect_median']*100,2))+'\%',' & ')

    
    #print(row['all']['country'].values)
    #for c,r in row['all'].values().iterrows():
        #
        
    if row['name']=='Lunchbox':
        for i in range(11):
            print('+'+str(round(row['all']['coeff'].values[i]*100,2))+'\%' if \
                  round(row['all']['coeff'].values[i],2) >=0 else \
                  str(round(row['all']['coeff'].values[i]*100,2))+'\%', make_stars(row['all']['p_val'].values[i]) + ' &')
    else:
        for i in range(12):
            print('+'+str(round(row['all']['coeff'].values[i]*100,2))+'\%' if \
                  round(row['all']['coeff'].values[i],2) >=0 else \
                  str(round(row['all']['coeff'].values[i]*100,2))+'\%', make_stars(row['all']['p_val'].values[i]) + ' &')

    print('----------------------------------')
    print('\n')

Take-out (Mode 2) & 
+154.59\%  & 
+146.02\% * &
+265.64\%  &
+201.39\% * &
+54.52\%  &
+365.72\% * &
+2.57\%  &
+145.19\% * &
+37.96\%  &
+87.29\% * &
+163.16\% * &
+256.26\% * &
+354.73\% * &
----------------------------------


Recipe (Mode 1) & 
+88.18\%  & 
+59.63\% * &
+77.33\% * &
+84.56\% * &
+54.3\% * &
+36.72\% * &
+157.9\% * &
+140.41\% * &
+170.7\% * &
+180.0\% * &
+122.57\% * &
+80.65\% * &
+91.8\% * &
----------------------------------


Food delivery (Mode 2) & 
+87.17\%  & 
+27.28\%  &
+834.23\%  &
+106.0\% * &
+90.29\%  &
-100.0\% * &
-22.33\%  &
+14883.59\%  &
+84.05\%  &
+27.49\%  &
+204.11\%  &
+241821.81\%  &
+65.38\% * &
----------------------------------


Baking (Mode 1) & 
+84.23\%  & 
+68.69\% * &
+148.33\% * &
+72.61\% * &
+84.22\% * &
+16.2\%  &
+393.1\% * &
+47.5\%  &
+175.09\% * &
+225.34\% * &
+85.96\% * &
+78.53\% * &
+84.24\% * &
----------------------------------


Cooking (Mode 1) & 
+59.38\%  & 
+30.67\% * &
+42.13\% * &
+34.99\% * &
+33.81\%  &
+75.